In [1]:
results_folder = "./results/"
data_folder = "./data/"
models_folder = "./models/"

In [2]:
import os

from DeepPurpose.dataset import *
import DTI_inspire as models
import matplotlib.pyplot as plt
import pandas as pd
from utils_inspire import *

In [3]:
X_drug, X_target, y  = load_process_DAVIS(path = data_folder, binary = False, convert_to_log = True, threshold = 30)

if not os.path.isfile(data_folder + 'BindingDB_All.tsv'):
    bdb_path = download_BindingDB(path = data_folder)
else:
    bdb_path = data_folder + 'BindingDB_All.tsv'
X_drug2, X_target2, y2  = process_BindingDB(path = bdb_path, df = None, y = 'Kd', binary = False, convert_to_log = True, threshold = 30)

Beginning Processing...
Beginning to extract zip file...
Default set to logspace (nM -> p) for easier regression
Done!
Loading Dataset from path...
b'Skipping line 772572: expected 193 fields, saw 205\nSkipping line 772598: expected 193 fields, saw 205\n'
b'Skipping line 805291: expected 193 fields, saw 205\n'
b'Skipping line 827961: expected 193 fields, saw 265\n'
b'Skipping line 1231688: expected 193 fields, saw 241\n'
b'Skipping line 1345591: expected 193 fields, saw 241\nSkipping line 1345592: expected 193 fields, saw 241\nSkipping line 1345593: expected 193 fields, saw 241\nSkipping line 1345594: expected 193 fields, saw 241\nSkipping line 1345595: expected 193 fields, saw 241\nSkipping line 1345596: expected 193 fields, saw 241\nSkipping line 1345597: expected 193 fields, saw 241\nSkipping line 1345598: expected 193 fields, saw 241\nSkipping line 1345599: expected 193 fields, saw 241\n'
b'Skipping line 1358864: expected 193 fields, saw 205\n'
b'Skipping line 1378087: expected 193

In [4]:
X_drug = np.concatenate((X_drug, X_drug2), axis=0)
X_target = np.concatenate((X_target, X_target2), axis=0)
y = np.concatenate((y, y2), axis=0)

In [5]:
upper = np.vectorize(lambda x: x.upper())
X_drug = upper(X_drug)
X_target = upper(X_target)

In [6]:
all_data = pd.DataFrame(data=np.transpose(np.stack([X_drug, X_target, y])))
all_data

,0,1,2
0,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,7.3655227298392685
1,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,4.999995657076895
2,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,4.999995657076895
3,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,4.999995657076895
4,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,4.999995657076895
...,...,...,...
96495,CC(C)(C)C[C@@H]1N[C@H]([C@H](C2CCCC(CL)C2F)[C@...,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,8.55284196865778
96496,CC[C@@H]([C@H](C)O)N1[C@@H]([C@H](C[C@](C)(CC(...,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,9.301029995663981
96497,CC(C)[C@@H](CS(=O)(=O)C(C)C)N1[C@@H]([C@H](C[C...,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,9.838631997765026
96498,CCCCCCCOC1OC(CO)C(O)C(O)C1O,MKRVITLFAVLLMGWSVNAWSFACKTANGTAIPIGGGSANVYVNLA...,7.767003889607846


In [7]:
# Keep DAVIS version if duplicates exist
all_data = all_data.drop_duplicates(subset=[0, 1], keep='first')
all_data

,0,1,2
0,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,7.3655227298392685
1,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,4.999995657076895
16,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,MVLGTVLLPPNSYGRDQDTSLCCLCTEASESALPDLTDHFASCVED...,4.999995657076895
17,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,MVDGVMILPVLIMIALPSPSMEDEKPKVNPKLYMCVCEGLSCGNED...,4.999995657076895
18,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,MAESAGASSFFPLVVLLLAGSGGSGPRGVQALLCACTSCLQANYTC...,4.999995657076895
...,...,...,...
96492,O=C(NCCCCS(=O)(=O)C1CCCCC1)C1CCC2NCCN2C1,MNPAAEAEFNILLATDSYKVTHYKQYPPNTSKVYSYFECREKKTEN...,6.065451052333853
96493,CCOC1CC(CCC1C1=N[C@@](C)(C2CCC(CL)CC2)[C@](C)(...,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,8.522878745280337
96494,COC1CC(CCC1NC(=O)[C@@H]1N[C@@H](CC(C)(C)C)[C@@...,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,9.602059991327963
96495,CC(C)(C)C[C@@H]1N[C@H]([C@H](C2CCCC(CL)C2F)[C@...,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,8.55284196865778


In [8]:
drug_encoding, target_encoding = 'Transformer', 'CNN_inspire'
#ensure none of test smiles appears in set
train, val, test  = data_process(all_data[0].to_numpy(), all_data[1].to_numpy(), y, 
                      drug_encoding, target_encoding, 
                      split_method='cold_protein',frac=[0.8,0.1,0.1],
                      cnn_inspire_use_transformer_embedding=True,
                      random_seed = 42)
# train.head()
print(f"First drug representation: \n{train.drug_encoding.iloc[0]}")
print(f"First target representation: \n{train.target_encoding.iloc[0]}")

Drug Target Interaction Prediction Mode...
in total: 71845 drug-target pairs
encoding drug...
unique drugs: 10685
encoding protein...
unique target sequence: 1449
splitting dataset...
Done.
First drug representation: 
(array([ 800,  122,  248,  282,  623,  272, 1256, 2210,   91,   85,  109,
        119,   80,    8,  282,  861,  209,   19,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0]), array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0]))
First target representation: 
[ 839  167  106   61  277  447  291 2515   32  447   94  291   45 1607
  200   87   67 1491   69  495 2820   32 1251  677   84   94   81  227
 1684  110  301   97  121 1429 3640  927  144 3875 3718 2624  558 1464
   33   92  320   33  319 31

In [9]:
train.to_pickle(data_folder + 'G35_data_train_t_emb.pkl')
val.to_pickle(data_folder + 'G35_data_vali_t_emb.pkl')
test.to_pickle(data_folder + 'G35_data_test_t_emb.pkl')